## Necessary libraries

In [ ]:
import gsd.hoomd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import gspread

In [ ]:
def compute_all_flake_coms(flake_positions, box_lengths, beads_per_flake=50):
    Lx, Ly, Lz = box_lengths
    n_flakes = len(flake_positions) // beads_per_flake
    flake_coms = []

    for i in range(n_flakes):
        flake = flake_positions[i * beads_per_flake : (i + 1) * beads_per_flake]
        ref = flake[0]
        delta = flake - ref

        # Minimum image convention
        delta[:, 0] -= Lx * np.round(delta[:, 0] / Lx)
        delta[:, 1] -= Ly * np.round(delta[:, 1] / Ly)
        delta[:, 2] -= Lz * np.round(delta[:, 2] / Lz)

        unwrapped = ref + delta
        com = np.mean(unwrapped, axis=0)

        # Wrap CoM into [-L/2, L/2]
        com[0] = (com[0] + Lx/2) % Lx - Lx/2
        com[1] = (com[1] + Ly/2) % Ly - Ly/2
        com[2] = (com[2] + Lz/2) % Lz - Lz/2

        flake_coms.append(com)

    return np.array(flake_coms)
    

In [ ]:
def count_clusters(flake_coms, box_lengths, cutoff, include_singletons=True):
    Lx, Ly, Lz = box_lengths
    n = len(flake_coms)
    adjacency = [set() for _ in range(n)]

    for i in range(n):
        for j in range(i + 1, n):
            dr = flake_coms[j] - flake_coms[i]
            dr[0] -= Lx * np.round(dr[0] / Lx)
            dr[1] -= Ly * np.round(dr[1] / Ly)
            dr[2] -= Lz * np.round(dr[2] / Lz)
            if np.linalg.norm(dr) < cutoff:
                adjacency[i].add(j)
                adjacency[j].add(i)

    visited = set()
    clusters = []

    for i in range(n):
        if i not in visited:
            cluster = []
            stack = [i]
            while stack:
                cur = stack.pop()
                if cur not in visited:
                    visited.add(cur)
                    cluster.append(cur)
                    stack.extend(adjacency[cur] - visited)

            if include_singletons or len(cluster) > 1:
                clusters.append(cluster)

    return len(clusters), [len(c) for c in clusters], clusters


In [ ]:
def analyze_clusters_over_time(traj, start=0, step=1000, cutoff=2.5, beads_per_flake=50):
    times = []
    n_clusters_list = []
    avg_cluster_sizes = []

    for frame_idx in range(start, len(traj), step):
        frame = traj[frame_idx]
        box_lengths = frame.configuration.box[:3]

        flake_type = "F"
        flake_typeid = frame.particles.types.index(flake_type)
        positions = frame.particles.position
        typeids = frame.particles.typeid
        flake_positions = positions[typeids == flake_typeid]

        flake_coms = compute_all_flake_coms(flake_positions, box_lengths, beads_per_flake)
        n_clusters, sizes, _ = count_clusters(flake_coms, box_lengths, cutoff)

        times.append(frame_idx)
        n_clusters_list.append(n_clusters)
        avg_cluster_sizes.append(np.mean(sizes) if sizes else 0)

    return times, n_clusters_list, avg_cluster_sizes

# Adjust to whichever file you're trying to analyze, then hit run until the end

In [ ]:
name_start = "140_10mer5f_0.005dt_1.0kT_0.001id_0.16fd_500000.0ssteps_1000000.0rsteps"
start_file = f"{name_start}_start.txt"
traj_file = f"{name_start}.gsd"
with open(start_file) as f:
    print(f.readlines()[0].strip())

In [ ]:
with open(start_file) as f:
    start = int(f.readlines()[0].strip()) # extracting post-shrink sim beginning frame
traj = gsd.hoomd.open(traj_file)
times, n_clusters, avg_sizes = analyze_clusters_over_time(
    traj,
    start=start,   # post-shrink
    step=1, # analyze every "x" frame, leave this at 1
    cutoff=2.5
) # this function is used to initially calculate the decorrelation time, so that we know every how many certain amount of frames to analyze. 

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(times, n_clusters, marker='o')
plt.title("Number of Clusters Over Time")
plt.xlabel("Frame")
plt.ylabel("Cluster Count")

plt.subplot(1, 2, 2)
plt.plot(times, avg_sizes, marker='s', color='orange')
plt.title("Average Cluster Size Over Time")
plt.xlabel("Frame")
plt.ylabel("Avg. Cluster Size")

plt.tight_layout()
plt.show()

In [ ]:
def autocorr1D(array):
    ft = np.fft.rfft(array - np.average(array))  
    acorr = np.fft.irfft(ft * np.conjugate(ft))  
    acorr /= (len(array) * np.var(array))        

    dt = np.where(acorr < 0)[0][0]               
    nsamples = len(array) // dt                  

    acorr = acorr[dt:nsamples]                  

    return nsamples, dt
n, dt = autocorr1D(avg_sizes)

# Decorrelation time is now calculated; now we can plot all the independent samples present within this simulation. Lower independent samples means more average aggregation over time is occurring, should occur as kT is increased. 

In [ ]:
print("Independent samples and decorrelation time:", n, dt)

In [ ]:
# now we can plot every "dt"th frame, whose length should be ~= independent samples
decorrelated_times = times[::dt]
decorrelated_sizes = avg_sizes[::dt]
decorrelated_clusters = n_clusters[::dt]

In [ ]:
print(len(decorrelated_sizes)) # verifying it is equal to calculated independent sampleses))

In [ ]:
plt.plot(decorrelated_times, decorrelated_sizes) # plot of every independent sample in simulation
plt.title("Average Cluster Size (Decorrelated)")
plt.xlabel("Frame")
plt.ylabel("Avg Cluster Size")

In [ ]:
np.mean(decorrelated_sizes)

In [ ]:
np.mean(decorrelated_clusters)

In [ ]:
plt.plot(decorrelated_times, decorrelated_clusters)
plt.title("Number of Clusters (Decorrelated)")
plt.xlabel("Frame")
plt.ylabel("# Clusters")

In [ ]:
'''# Add additional data to start_file
decorrelated_values = [len(decorrelated_sizes), np.mean(decorrelated_sizes), np.mean(decorrelated_clusters)]
values_to_add = [str(val) + '\n' for val in decorrelated_values]
with open(start_file, 'a') as f:
    f.write(str(len(decorrelated_sizes)) + '\n')
    f.write(str(f"{np.mean(decorrelated_sizes):.2f}") + '\n')
    f.write(str(f"{np.mean(decorrelated_clusters):.2f}") + '\n')'''

In [ ]:
'''# Add to Google Sheet
service_account_file = "../service_worker1.json"

gc = gspread.service_account(filename=service_account_file)

spreadsheet = gc.open("Mxene Project Data")
worksheet = spreadsheet.worksheet("Week 11 Data")
data = worksheet.get_all_values()
print(f"Successfully read {len(data)} rows from the sheet.")

with open(start_file, 'r') as f:
    new_row_data = f.readlines()[1:]

worksheet.append_row(new_row_data)'''